In [ ]:
# first we define relevant directories
import sys
import os
import copy
# project directory
project_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
# src directory 
src_dir = os.path.join(project_dir, 'src')
sys.path.insert(0, src_dir)
print("src_dir ", src_dir)

In [ ]:
from train_eval import *

In [ ]:
NUM_PROCESSORS = mp.cpu_count() - 2

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '' # specify the visibile devices (if preferred)

In [ ]:
def optimize_hyperparams(dataset_dir, dsettypes, wrk_dir, model_class,
                         num_epochs, to_gpu, memmap, options={}): 
    # get parameter configurations
    prob_interval_truemax=0.03
    prob_estim = 0.95
    rnn_class = True
    crf_mode = None
    target_fold = "subsetfold_none"
    if(model_class == 'RNN_Labeler'):
        alpha_param = True
        sampling_func = False
        y_codebook = {0:0, 1:1}
        model_train = rnn_run
    elif(model_class ==  'RNNSS_Labeler'):
        alpha_param = True
        sampling_func = True
        y_codebook = {0:0, 1:1, '__START__':2}
        model_train = rnnss_run
    elif(model_class in {'CRF_Labeler', 'CRF_Pair_Labeler', 
                         'RNNCRF_Pair_Labeler', 'RNNCRF_Unary_Labeler'}):
        alpha_param = False
        sampling_func = False
        y_codebook = {0:0, 1:1, '__START__':2}
        model_train = rnncrf_run
        if(model_class == 'CRF_Labeler'):
            crf_type = options.get('crf_type') # {nn_crf, only_crf}
            rnn_class = False
            if(crf_type == 'nn_crf'):
                crf_mode = 'crf'
            else:
                crf_mode = 'crf_nn'
        stopstate_symb = options.get('stopstate_symb')
        if(stopstate_symb):
            y_codebook[stopstate_symb] = len(y_codebook)
    elif(model_class in {'CNN_Labeler', 'CNNWide_Labeler', 'NN_Labeler'}):
        y_codebook = {0:0,1:1}
        model_train = cnn_nn_run
        options['cnn_type'] = model_class
        
    classweight_option = options.get('classweight_option')
    target_dsets, input_shape = load_dataset(os.path.join(dataset_dir, target_fold, 'dataset_tuple.pkl'),
                                             dsettypes, 
                                             memmap)
    if(classweight_option):
        class_weights = ReaderWriter.read_data(os.path.join(dataset_dir,
                                                            target_fold,
                                                            'train_pdtm', 
                                                            'classweights_'+classweight_option+'.pkl'))
        options['class_weights'] = class_weights    

    if(model_class in {'CNN_Labeler', 'CNNWide_Labeler'}):
        # C: channels, H: y-dimension of the matrix, W: x-dimension of the matrix
        C, H, W = 1, input_shape[-2], input_shape[-1]
        configs= get_config_cnn((C, H, W), model_class, prob_interval_truemax, prob_estim)
    elif(model_class == 'NN_Labeler'):
        input_dim = input_shape[-1] # number of input features
        configs = get_config_nn(input_dim, prob_interval_truemax, prob_estim)
    else:
        input_dim = input_shape[-1] # number of input features
        configs = get_config(input_dim, prob_interval_truemax, prob_estim, 
                             alpha_param=alpha_param, sampling_func=sampling_func,
                             rnn_class=rnn_class, crf_mode=crf_mode)
    args = []
    model_dirs = []
    for i, config in enumerate(configs):
        # create a new version for each configuration
        target_dsets, __ = load_dataset(os.path.join(dataset_dir, target_fold, 'dataset_tuple.pkl'),
                                        dsettypes, 
                                        memmap)
        # setup target dir
        target_dir = create_directory('out_{}'.format(i), wrk_dir)
        model_dirs.append(target_dir)
        args.append((config, model_class, target_dsets, y_codebook, num_epochs,
                     target_dir, i, to_gpu, memmap, None, options))
    # pickle the configs
    ReaderWriter.dump_data(configs, os.path.join(wrk_dir, 'configs.pkl'))
    num_processors = mp.cpu_count()-1
    num_models = len(args)
    outlog = os.path.join(wrk_dir, 'out.log')

    if(num_processors > NUM_PROCESSORS):
        num_processors = NUM_PROCESSORS
    if(num_models < num_processors):
        num_processors = num_models
    print("number of models to be evaluted: ", num_models)
    print("number of processors to be used: ", num_processors)
    
    processes = []
    counter = 0
    tick = datetime.datetime.now()
    while len(args):
        if(len(args) < num_processors):
            num_processors = len(args)
        for __ in range(num_processors):
            arg = args.pop()
            p = mp.Process(target=model_train, args=arg)
            p.start()
            processes.append(p)
            print("counter ", counter)
            counter+=1
        for p in processes:
            p.join()
            print("we are joining {}".format(p))
    tock = datetime.datetime.now()
    line = "total number of seconds: {}\n".format((tock-tick).seconds)
    ReaderWriter.write_log(line, outlog, mode='a')
    scores = get_perfscores(model_dirs, 'validation', outlog, wrk_dir)
    return(scores)

In [ ]:
hyperparam_optimzmodel_dir =  create_directory('hyperparam_optimz_models', project_dir)
dataset_dir = os.path.join(project_dir, 'dataset_neural')

## RNN/RNNSS

In [ ]:
# checking __main__ condition helps preventing "weird" issues when running multiprocesses on Windows OS!!
if __name__ == "__main__":
    to_gpu = True # run on GPU
    memmap = True # data is memory mapped originally using numpy memmap
    bidirection = False
    classweight_option = 'last_indx'
    loss_mode = 'Convex_HF_LastHF'
    rgrad_mode = 'clip_norm'
    rgrad_limit = (None, 1) 
    model_classes = ('RNN_Labeler', 'RNNSS_Labeler')
    prefixes = ('rnn', 'rnnss')    
    # hyperparam optimization phase
    num_epochs = 25
    for i, model_class in enumerate(model_classes):
        print("model class: ", model_class)
        prefix = prefixes[i]
        wrk_dir = create_directory(prefix, hyperparam_optimzmodel_dir)
        scoredict = optimize_hyperparams(dataset_dir, ['train', 'validation'], 
                                         wrk_dir, model_class,
                                         num_epochs, to_gpu, memmap,
                                         {'classweight_option':classweight_option,
                                          'reg_type':'l2',
                                          'loss_mode':loss_mode,
                                          'bidirection':bidirection,
                                          'rgrad_mode':rgrad_mode,
                                          'rgrad_limit':rgrad_limit})
        print(scoredict)
        print("-"*25)

## CNN/CNNWide/MLP

In [ ]:
if __name__ == "__main__":    
    to_gpu = True
    memmap = True
    classweight_option = 'last_indx'    
    rgrad_mode = 'clip_norm'
    rgrad_limit = (None, 1)
    model_classes = ('CNN_Labeler', 'CNNWide_Labeler', 'NN_Labeler')
    prefixes = ('cnn', 'cnnwide', 'nn')
    num_epochs = 25
    for i, model_class in enumerate(model_classes):
        prefix = prefixes[i]
        wrk_dir = create_directory(prefix, hyperparam_optimzmodel_dir)
        scoredict = optimize_hyperparams(dataset_dir, ['train', 'validation'], 
                                         wrk_dir, model_class,
                                         num_epochs, to_gpu, memmap,
                                         {'classweight_option':classweight_option,
                                          'reg_type':'l2',
                                          'rgrad_mode':rgrad_mode,
                                          'rgrad_limit':rgrad_limit})
        print(scoredict)
        print("-"*25)

## RNNCRF {Pair, Unary}

In [ ]:
if __name__ == "__main__":
    to_gpu = True
    memmap = True
    bidirection = False
    decoder_type = 'viterbi'
    rgrad_mode = 'clip_norm'
    rgrad_limit = (None, 1)
    model_classes = ('RNNCRF_Pair_Labeler', 'RNNCRF_Unary_Labeler')
    prefixes = ('rnncrfpair', 'rnncrfunary')
    num_epochs = 25
    for i, model_class in enumerate(model_classes):
        prefix = prefixes[i]
        wrk_dir = create_directory(prefix, hyperparam_optimzmodel_dir)
        scoredict = optimize_hyperparams(dataset_dir, ['train', 'validation'], 
                                         wrk_dir, model_class,
                                         num_epochs, to_gpu, memmap,
                                         {'reg_type':'l2',
                                          'decoder_type':decoder_type,
                                          'bidirection':bidirection,
                                          'rgrad_mode':rgrad_mode,
                                          'rgrad_limit':rgrad_limit})
        print(scoredict)
        print("-"*25)

## CRF/Neural CRF {Pair, Unary}

In [ ]:
if __name__ == "__main__":
    to_gpu = True
    memmap = True
    decoder_type = 'viterbi'
    rgrad_mode = 'clip_norm'
    rgrad_limit = (None, 1)
    model_classes = ('CRF_Pair_Labeler', 'CRF_Labeler')
    crf_types = ('nn','only')
    num_epochs = 25
    for model_class in model_classes:
        for crf_type in crf_types:
            if('Pair' in model_class):
                prefix = 'crf{}pair'.format(crf_type)
            else:
                prefix = 'crf{}'.format(crf_type)
            wrk_dir = create_directory(prefix, hyperparam_optimzmodel_dir)
            scoredict = optimize_hyperparams(dataset_dir, ['train', 'validation'], 
                                             wrk_dir, model_class,
                                             num_epochs, to_gpu, memmap,
                                             {'reg_type':'l2',
                                              'decoder_type':decoder_type,
                                              'crf_type': '{}_crf'.format(crf_type),
                                              'bidirection':bidirection,
                                              'rgrad_mode':rgrad_mode,
                                              'rgrad_limit':rgrad_limit})
            print(scoredict)
            print("-"*25)